In [1]:
import torch
from torch import nn
from torch.autograd import Variable

from data_loader import DataLoader
from model import FillThoughts
from config import *
from datetime import datetime, timedelta

from tensorboardX import SummaryWriter
import nltk

from vocab import load_dictionary

import sys
sys.path.append('..')
import gather
from sacremoses import MosesDetokenizer
detokenizer = MosesDetokenizer()

import numpy as np

[nltk_data] Downloading package punkt to /jet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
d = DataLoader(sentences=[''], word_dict=load_dictionary('../dataset/english/all.tokenized.txt.pkl'))
save_loc = "./saved_models/skip-best-0.0003-20000"
model = FillThoughts()
if USE_CUDA:
    model.cuda(CUDA_DEVICE)
model.load_state_dict(torch.load(save_loc))

Making reverse dictionary


/jet/var/python/lib/python3.6/site-packages/torch/backends/cudnn/__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [3]:
def std(sent):
    return " ".join(nltk.word_tokenize(sent))

def infer(before, after, hint=None):
    before = std(before)
    after = std(after)
    ind_before = d.convert_sentence_to_indices(before).unsqueeze(dim=0)
    ind_after = d.convert_sentence_to_indices(after).unsqueeze(dim=0)
    if hint is None:
        input_tensor = torch.cat([ind_before, ind_before, ind_after])
    else:
        hint = std(hint)
        ind_hint = d.convert_sentence_to_indices(hint).unsqueeze(dim=0)
        input_tensor = torch.cat([ind_before, ind_hint, ind_after])
    thoughts, emb = model.encoder(input_tensor)
    if hint is None:
        pred = model.decoder(thoughts, None, model.encoder.word2embd)
    else:
        pred = model.decoder(thoughts, emb)
        
    _, pred = pred.max(dim=2)
    res = d.convert_indices_to_sentences(pred[0])
    return detokenizer.detokenize(res.split(" "), return_str=True)

In [10]:
# I shoved the pages under my shirt before I made it through the last door .
# The village was crowded with elves , oblivious to my horror 
# The protection spell must have only alerted council .


print(infer("You just met him in the morning.","Stared at.","Not even met."))

UNK like his his EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS


/jet/var/python/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/jet/var/python/lib/python3.6/site-packages/torch/backends/cudnn/__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [ ]:

import tqdm
sentences_vectors = None
pre_sentence = None
embedding_size = 1200
batch_size = 256

print("Pre-computing vectors")
path = "../dataset/english/all.tokenized.txt"
author_sentences = DataLoader(path)
n_sent = len(author_sentences.sentences)
print(n_sent)
n_sent = 1000*1000
sentences_vectors = np.empty((n_sent, embedding_size))
pre_sentence = []

for i in tqdm.tqdm(range(0, n_sent-batch_size, batch_size)):
    batch = []
    for j in range(i, min(i + batch_size, n_sent)):
        sent = author_sentences.sentences[j]
        pre_sentence.append(sent)
        ind = d.convert_sentence_to_indices(sent)
        batch.append(ind)
    output, _ = model.encoder(torch.stack(batch))
    sentences_vectors[i:min(i+batch_size, n_sent)] = output.cpu().data.numpy()

Pre-computing vectors
Loading text file at ../dataset/english/all.tokenized.txt
Making dictionary for these words
Using cached dictionary at ../dataset/english/all.tokenized.txt.pkl

  0%|          | 0/3906 [00:00<?, ?it/s]


Making reverse dictionary
2472079


/jet/var/python/lib/python3.6/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/jet/var/python/lib/python3.6/site-packages/torch/backends/cudnn/__init__.py:89: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "
 47%|████▋     | 1839/3906 [05:28<06:09,  5.60it/s]

In [26]:
import gather
import tqdm
from scipy.spatial import distance
from torch.nn import functional as F

def prepare_test(sentence):
    return " ".join(nltk.word_tokenize(sentence))

def get_vector(sentence):
    indices = d.convert_sentence_to_indices(sentence)
    output, _ = model.encoder(torch.stack([indices]))
    return output
    
def get_closest_sentence(sentence):
    target_vector = get_vector(prepare_test(sentence)).cpu().data.numpy()
    sentences = sentences_vectors
    
    max_sim = 0
    
    for i, vector in tqdm.tqdm(enumerate(sentences)):
        sim = 1 - distance.cosine(vector, target_vector)

        if sim > max_sim:
            max_sim = sim
            max_sent = pre_sentence[i]
    return max_sent, max_sim

In [11]:
ms, mss = get_closest_sentence("I'm from France.")
print(mss)
print(detokenizer.detokenize(ms.split(" "), return_str=True))

NameError: name 'get_closest_sentence' is not defined